In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#coding:utf-8
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os


In [0]:

# DSD100のデータは元から44.1kHzのみなので今回は指定なし
# SAMPLING_RATE = 44100 
FRAME_SIZE = 2048
# over_wrap = 50%
HOP_SIZE = 1024


def extract_feature_by_stft(file_path, n_fft=None, hop_length=None):
    npy_file_path = file_path.replace('.wav', '_stft')
    if os.path.exists(npy_file_path):
        return np.load(npy_file_path)
    y, sr = librosa.load(file_path)

    args = {}
    args['y'] = y
    if n_fft is not None:
    args['n_fft'] = n_fft
    if hop_length is not None:
    args['hop_length'] = hop_length

    D = np.abs(librosa.stft(**args))

    np.save(npy_file_path, D)
    return D


root_dir_path = "/content/drive/My Drive/Paper/MMDense-Net/"
dir_path_list = ["051 - AM Contra - Heart Peripheral/"]

for dir_path in dir_path_list:
    mixture_file_path = root_dir_path + "Mixtures/" + dir_path + "mixture.wav"
    source_file_path_dict = {}
    for i in range(4):
        key = ['bass', 'drums', 'other', 'vocals'][i]
        path = root_dir_path + "Sources/" + dir_path + key + ".wav"
        source_file_path_dict[key] = path
    # mix
    Mix_D = extract_feature_by_stft(mixture_file_path, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)

    # sources
    Sources_D_dict = dict(
        [[key, extract_feature_by_stft(value)] for key, value in  source_file_path_dict.items()]
    )  

In [0]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os


In [31]:

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

if not os.path.exists('./dc_img_only_data'):
    os.mkdir('./dc_img_only_data')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    transforms.Normalize((0.5,), (0.5,))
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset.data, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img_only_data/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './conv_autoencoder_only_data.pth')
# param = torch.load('model.pth')
# model = autoencoder() #読み込む前にクラス宣言が必要
# model.load_state_dict(param)

RuntimeError: ignored

In [0]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    transforms.Normalize((0.5,), (0.5,))
])

dataset = MNIST('./data', transform=img_transform, download=True)

In [5]:
dataset.data.shape

torch.Size([60000, 28, 28])

In [0]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3
dataloader = DataLoader(dataset.data, batch_size=batch_size, shuffle=True)

In [29]:
dataloader.__dict__

{'_DataLoader__initialized': True,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x7f5b77fd2320>,
 'batch_size': 128,
 'collate_fn': <function torch.utils.data._utils.collate.default_collate>,
 'dataset': tensor([[[5.8612e-04, 1.0012e-04, 4.2933e-04,  ..., 3.4408e-04,
           2.7148e-04, 2.7125e-04],
          [4.4804e-04, 4.8357e-05, 3.7096e-04,  ..., 3.0921e-04,
           2.3744e-04, 2.2167e-04],
          [2.5109e-04, 3.3724e-05, 3.4696e-04,  ..., 3.8694e-04,
           8.1846e-05, 1.5278e-04],
          ...,
          [5.2779e-06, 1.3963e-06, 1.7204e-06,  ..., 3.4277e-07,
           4.1070e-07, 2.7675e-05],
          [3.9800e-06, 1.4809e-06, 3.7638e-07,  ..., 5.6260e-07,
           4.0235e-07, 2.8816e-05],
          [3.0980e-06, 5.3938e-07, 2.4932e-07,  ..., 2.4536e-07,
           3.0605e-07, 2.9493e-05]],
 
         [[5.8612e-04, 1.0012e-04, 4.2933e-04,  ..., 3.4408e-04,
           2.7148e-04, 2.7125e-04],
          [4.4804e-04, 4.8357e-05, 3.7096e-04,  ..., 3.09

In [27]:
dataloader.__dict__

{'_DataLoader__initialized': True,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x7f5b77fd2240>,
 'batch_size': 128,
 'collate_fn': <function torch.utils.data._utils.collate.default_collate>,
 'dataset': tensor([[[5.8612e-04, 1.0012e-04, 4.2933e-04,  ..., 3.4408e-04,
           2.7148e-04, 2.7125e-04],
          [4.4804e-04, 4.8357e-05, 3.7096e-04,  ..., 3.0921e-04,
           2.3744e-04, 2.2167e-04],
          [2.5109e-04, 3.3724e-05, 3.4696e-04,  ..., 3.8694e-04,
           8.1846e-05, 1.5278e-04],
          ...,
          [5.2779e-06, 1.3963e-06, 1.7204e-06,  ..., 3.4277e-07,
           4.1070e-07, 2.7675e-05],
          [3.9800e-06, 1.4809e-06, 3.7638e-07,  ..., 5.6260e-07,
           4.0235e-07, 2.8816e-05],
          [3.0980e-06, 5.3938e-07, 2.4932e-07,  ..., 2.4536e-07,
           3.0605e-07, 2.9493e-05]],
 
         [[5.8612e-04, 1.0012e-04, 4.2933e-04,  ..., 3.4408e-04,
           2.7148e-04, 2.7125e-04],
          [4.4804e-04, 4.8357e-05, 3.7096e-04,  ..., 3.09

In [0]:
D = np.load('/content/drive/My Drive/Paper/MMDense-Net/Mixtures/051 - AM Contra - Heart Peripheral/mixture_stft.npy')

In [14]:
D.shape

(1025, 4525)

In [0]:
data = np.tile(D, (600, 1, 1))

In [22]:
data.shape

(600, 1025, 4525)

In [0]:
data = torch.from_numpy(data)

In [24]:
data.shape

torch.Size([600, 1025, 4525])

In [0]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

In [36]:

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

if not os.path.exists('./dc_img'):
    os.mkdir('./dc_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

# img_transform = transforms.Compose([
#     transforms.ToTensor(),
# #     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     transforms.Normalize((0.5,), (0.5,))
# ])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class MyDataset(torch.utils.data.Dataset):

    def __init__(self, data_num, transform=None):
        self.transform = transform
        self.data_num = data_num
        self.data = []
        self.label = []
        for x in range(self.data_num):
            self.data.append(x) # 0 から (data_num-1) までのリスト
            self.label.append(x%2 == 0) # 偶数ならTrue 奇数ならFalse

    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label =  self.label[idx]

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, out_label


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

device = torch.device('cuda:0')
model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img)
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.data))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, './dc_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './MNIST_test.pth')
# param = torch.load('model.pth')
# model = autoencoder() #読み込む前にクラス宣言が必要
# model.load_state_dict(param)

epoch [1/100], loss:0.1843


KeyboardInterrupt: ignored

In [37]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

param = torch.load('/content/drive/My Drive/Paper/MNIST_CAE/conv_autoencoder.pth')
device = torch.device('cuda:0')
model = autoencoder() #読み込む前にクラス宣言が必要
model.load_state_dict(param)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
for data in dataloader:
    img, _ = data
    img = Variable(img)
    predict_img = model(img)
    break

In [59]:
predict_img

tensor([[[[-0.9991, -0.9987, -0.9993,  ..., -0.9999, -1.0000, -0.9974],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -0.9999, -1.0000, -1.0000]]],


        [[[-0.9880, -0.9862, -0.9976,  ..., -0.9995, -0.9999, -0.9963],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [-0.9999, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-0.9999, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  ..., -0.9998, -1.0000, -1.0000]]],


        [[[-0.9994, -0.9998, -0.9999,  ..., -0.9999, -1.0000, -0.9964],
          [-1.0000, -1.000

In [60]:
output.cpu().data

tensor([[[[-0.9887, -0.9955, -0.9988,  ..., -0.9998, -0.9999, -0.9981],
          [-0.9989, -0.9969, -0.9996,  ..., -1.0000, -1.0000, -0.9999],
          [-0.9982, -0.9969, -0.9998,  ..., -0.9999, -1.0000, -0.9995],
          ...,
          [-0.9989, -0.9921, -0.9974,  ..., -0.9994, -0.9967, -0.9984],
          [-0.9996, -0.9960, -0.9989,  ..., -0.9937, -0.9897, -0.9924],
          [-1.0000, -0.9986, -0.9998,  ..., -0.9994, -0.9991, -0.9838]]],


        [[[-0.9996, -0.9998, -1.0000,  ..., -0.9999, -1.0000, -0.9997],
          [-1.0000, -0.9999, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-1.0000, -0.9999, -1.0000,  ..., -1.0000, -1.0000, -0.9999],
          ...,
          [-0.9995, -0.9944, -0.9982,  ..., -0.9999, -0.9991, -0.9987],
          [-0.9998, -0.9975, -0.9994,  ..., -0.9984, -0.9958, -0.9902],
          [-1.0000, -0.9993, -0.9999,  ..., -0.9999, -0.9997, -0.9823]]],


        [[[-0.9967, -0.9991, -0.9998,  ..., -0.9999, -1.0000, -0.9993],
          [-0.9998, -0.999

In [43]:
pic = to_img(output.cpu().data)
save_image(pic, './dc_img/image_)

RuntimeError: ignored